In [445]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pylab as plt

import pandas as pd
import numpy as np
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [357]:
#Load staff data
file_name = 'West_Way_Accreditation/20161207 FY16 West Way Accreditation Report.xlsx'

xl_file = pd.ExcelFile(file_name)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

#Save DF for each course
df_SC = dfs['SC Report']
df_T = dfs['T Report']
df_SA = dfs['SA Report']
df_MT = dfs['MT Report']

In [358]:
#Load target data
targets = 'West_Way_Accreditation/FY16 West Way Dealer Accreditation Targets.xlsx'

xl_file = pd.ExcelFile(targets)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

#Save DF for each course
df_targets = dfs['FY16 Targets']
df_targets.fillna(0,inplace=True)

In [359]:
#Adjusting the headers 
df_SC.columns = df_SC.iloc[[4]].values[0]
df_SC.drop(df_SC.index[0:5],inplace=True)

#Adjusting the headers 
df_T.columns = df_T.iloc[[4]].values[0]
df_T.drop(df_T.index[0:5],inplace=True)

#Adjusting the headers 
df_SA.columns = df_SA.iloc[[4]].values[0]
df_SA.drop(df_SA.index[0:5],inplace=True)

#Adjusting the headers 
df_MT.columns = df_MT.iloc[[4]].values[0]
df_MT.drop(df_MT.index[0:5],inplace=True)

In [360]:
#Drop common surplus headers
surplus_headers_SC = ['Current Accreditation Status', 'Completed/Booked all F2F criteria ',
           'Completed/Booked all Live criteria ', 'SC PDP ','Initial e-Learning ',
           'Annual Face to Face ', 'Annual VCT', 'Annual e-Learning ']

surplus_headers_T = ['Current Accreditation Status', 'Completed/Booked F2F criteria ',
                    'Completed/Booked all live criteria', 'Induction', 'e-Learning ',
                    'Annual e-Learning']

surplus_headers_SA = ['Current Accreditation Status', 'Completed/Booked F2F criteria ',
           'Completed/Booked all live criteria ', 'SA PDP ','e-Learning ',
           'Annual Face to Face ', 'Annual e-Learning ']

surplus_headers_MT = ['Completed/Booked F2F criteria ','Current Accreditation Status',
                      'Completed/Booked all live criteria','MT Assessment', 'e-Learning ',
                      'Annual Update ', 'Annual e-Learning']

df_SC.drop(surplus_headers_SC,1,inplace=True)
df_T.drop(surplus_headers_T,1,inplace=True)
df_SA.drop(surplus_headers_SA,1,inplace=True)
df_MT.drop(surplus_headers_MT,1,inplace=True)

In [361]:
def aggregate_course(df):
    '''
    This function cleans up the data for a given accreditation criteria.
    Valuing completed coures as 1 and booked courses and 0.5. 
    Each course is weighted by the total number for that criteria and summed to give a fraction.
    The DataFrame is then update.
    '''
    #replace categoricals with numeric values
    df.replace('Complete',1.0,inplace=True)
    df.replace('Booked',0.5,inplace=True)
    df.fillna(0,inplace=True)
        
    #wieghting
    total_course_fraction = []
    total_course_completion = []
    l = len(df)
    
    for i in range(l):
        course_completion = df.iloc[[i]].values[0][5:-3]
        total_courses = len(course_completion)
        course_fraction = sum(map(lambda x: x/total_courses, course_completion))
        total_course_fraction.append(course_fraction)
        if course_fraction > 0.999:
            total_course_completion.append(1)
        else:
            total_course_completion.append(0)

    #Update Dataframe
    df['Total fraction'] = pd.Series(total_course_fraction, index=df.index)
    df['Total completion'] = pd.Series(total_course_completion, index=df.index)
    
    return df


In [362]:
df_SC = aggregate_course(df_SC)
df_T = aggregate_course(df_T)
df_SA = aggregate_course(df_SA)
df_MT = aggregate_course(df_MT)

In [425]:
df_SA.groupby('Code').get_group(1877)

,Username,First Name,Last Name,Code,Dealership,Service Advisor Training Needs Analysis,Nissan Service 1,Nissan Service 2,Nissan Products and Systems,VCT: The Art of Communicating Value,...,Get the most out of NP300 Navara Part 3: Electrical,Quality: Mindfulness at Work,Quality: Service Appointment,Quality: Service Greeting and Consulting,Quality: Service Vehicle Delivery,Service NSSW: Customer Quality in Action,All New Micra Part 1,FY16 Service Advisor Knowledge Retention Assessment (Coming Soon),Total fraction,Total completion
6,230407,Bhupendra,Patel,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.965517,0
8,231786,Zach,Smart,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.000000,1
18,225737,Vishram,Gami,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.948276,0


In [417]:
#Count people who completed the criteria and calculation a dealership completion fraction

#Deffine the list of dealers
dealers = []
for i in df_SC['Code'].unique():
    dealers.append(i)
    
#Deffine the criteria
criteria = [df_MT,df_SC,df_SA,df_T]

#Calculate:
#a) the number of people on the course, 
#b) number of completions on the course,
#c) fraction of people who have completed across a dealer.

total_completion_dict = {}
total_completion_results = []

for i in dealers:
    people_total = 0
    completed_total = 0
    for j in criteria:
        try:
            dg = j.groupby('Code').get_group(i)
            people = len(dg['Total completion'])
            completed = len(dg.loc[dg['Total completion'] == 1.0])
            completion_frac = round(float(completed)/float(people),2)
        except:
            people = np.nan
            completed = np.nan
            completion_frac = np.nan
            
        total_completion_results.append(completion_frac)
        people_total = people_total + people
        completed_total = completed_total + completed
    
    total_completion_results.append(round(np.nansum(float(completed_total))/np.nansum(float(people_total)),2))
    total_completion_dict[i] = total_completion_results
    total_completion_results = []

#Produce DataFrame with results    
df_completion = pd.DataFrame(data=total_completion_dict)
df_completion = df_completion.T
df_completion.columns=['MT','SC','SA','T','All']
df_completion

,MT,SC,SA,T,All
1525,NaN,0.00,0.00,0.0,NaN
1533,0.0,0.00,0.00,0.0,0.00
1535,1.0,0.20,0.00,0.0,0.21
1633,1.0,0.00,0.00,0.0,0.06
1650,0.0,0.00,0.00,0.0,0.00
1651,0.0,0.86,0.00,0.0,0.32
1655,0.5,0.13,0.00,0.0,0.13
1692,1.0,0.13,0.00,0.0,0.10
1693,0.0,0.00,0.00,0.0,0.00
1694,0.0,0.00,0.00,0.0,0.00


In [442]:
#Deffine a list of dealers and filter targets
mean_list = []
sum_list = []
mean_dict = {}

for i in dealers:
    #print i
    for j in criteria:
        try:
            dg = j.groupby('Code').get_group(i)
            mean = round(np.mean(dg['Total fraction'].values),2)
            sums = np.nansum(dg['Total fraction'].values)
            mean_list.append(mean)
        except:
            mean_list.append(np.nan)
    sum_list.append(sums)
    #mean_list.append(sum(sum_list)/float(len(sum_list)))
    #print mean_list
    mean_dict[i] = mean_list
    mean_list = []


df_completion2 = pd.DataFrame(data=mean_dict)
df_completion2 = df_completion2.T
df_completion2.columns=['MT','SC','SA','T']
df_completion2

,MT,SC,SA,T
1525,NaN,0.69,0.45,0.19
1533,0.75,0.28,0.22,0.40
1535,1.00,0.77,0.21,0.55
1633,1.00,0.57,0.39,0.28
1650,0.69,0.62,0.72,0.50
1651,0.83,1.00,0.72,0.54
1655,0.76,0.45,0.83,0.20
1692,1.00,0.34,0.50,0.49
1693,0.93,0.61,0.59,0.50
1694,0.90,0.50,0.33,0.22


In [452]:
#sns.distplot(df_SC['Total fraction'],bins=25)
#plt.title('Distribution of celebrity death age (2005-2016)')
#plt.ylabel('Frequency')
#plt.xlabel('Fraction ')
